Perform MNIST classification using Neural Netwroks and Convolutional Neural Networks.

1) Use 10 iterations for training


2) Show the training loss for both networks on the same plot


3) Compare the test loss and accuracy.

In [3]:
!pip install tensorboard


     |████████████████████████████████| 5.9 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 17.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 30.1 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 17.4 MB/s eta 0:00:01


In [4]:
#preparations
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models,transforms
from torchvision.utils import make_grid
from torchvision.datasets import MNIST
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

# MNIST
def mnist(batch_sz, valid_size=0.2, shuffle=True, random_seed=2000):
    num_classes = 10
    transform_train = transforms.Compose([
                        transforms.RandomCrop(28, padding=4),
                        transforms.ToTensor(),
                    ])
    
    transform_valid = transforms.Compose([
                        transforms.ToTensor(),
                    ])
    
    transform_test = transforms.Compose([
                        transforms.ToTensor(),
                    ])
    

    # Training dataset
    train_data = MNIST(root='./datasets', train=True, download=True, transform=transform_train)
    valid_data = MNIST(root='./datasets', train=True, download=True, transform=transform_valid)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_sz, sampler=train_sampler,pin_memory=True)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_sz, sampler=valid_sampler,pin_memory=True)

    # Test dataset
    test_data = MNIST(root='./datasets', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_sz, shuffle=False, pin_memory=True)

    return train_loader, valid_loader, test_loader

In [5]:
batch_sz=64 # this is batch size i.e. the number of rows in a batch of data
train_loader, valid_loader, test_loader=mnist(batch_sz) 

In [6]:
class CNN(nn.Module):
  def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 10, 5, 1)
      self.conv2 = nn.Conv2d(10, 50, 5, 1)
      self.linear1 = nn.Linear(4 * 4 * 50, 100)
      self.linear2 = nn.Linear(100, 10)
      self.maxpool = nn.MaxPool2d((2, 2))

  def forward(self, x):
    #feature extracter
    x = self.conv1(x)
    x = F.relu(x)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.maxpool(x)

    x = x.view(-1, 4*4*50)

    #classifier
    x = self.linear1(x)
    x = F.relu(x)
    x = self.linear2(x)
    return(x)

In [7]:
class NN(nn.Module):
  def __init__(self, ni, nh, no):
    super().__init__()
    self.l1 = nn.Linear(ni, nh)
    self.l2 = nn.Linear(nh, no)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = self.l1(x)
    x = torch.sigmoid(x)
    x = self.l2(x)
    return x

ni = 28*28
nh = 100
no = 10

lr1 = 0.5
mm1 = 0.5
epochs1 = 10

ls1 = []

device1 = torch.device("cuda:0")
net1 = NN(ni, nh, no)
net1 = net1.to(device1)

optimizer = optim.SGD(net1.parameters(), lr = lr1, momentum = mm1)

loss_fn = nn.CrossEntropyLoss()
 
for i in range(epochs1):
  total_loss = 0
  for batch in train_loader:
    X, y = batch[0].to(device1), batch[1].to(device1)
    logits = net1(X)
    loss = loss_fn(logits, y)
    total_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  ls1.append(total_loss)


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
device = torch.device("cuda:0")
net = CNN()
net = net.to(device)

epochs = 10
lr = 0.1
mm = 0.5

ls = []
optimizer = optim.SGD(net.parameters(), lr = lr, momentum = mm)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.9, verbose = False)
loss_fn = nn.CrossEntropyLoss()

for i in range(epochs):
  total_loss = 0
  for batch in train_loader:
    X, y = batch[0].to(device),batch[1].to(device)
    logits = net(X)
    loss = loss_fn(logits, y)
    total_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  scheduler.step()
  ls.append(total_loss)


In [ ]:
plt.plot(ls1)
plt.plot(ls)

The picture above shows the total loss of NN and CNN.

In [ ]:
print("The loss of NN:")
print(ls1)
print("The loss of CNN:")
print(ls)

From the data above we can see the loss of CNN is much better than NN.

In [ ]:
!pip install torchmetrics

In [ ]:
import torchmetrics as tm
acc1 = []
accuracy1 = tm.Accuracy()
for batch in test_loader:
  X, y = batch[0].to(device), batch[1].to(device)
  logits = net1(X)
  yhat = torch.argmax(logits, axis = 1)
  a = accuracy1(yhat.to("cpu"), y.to("cpu"))
  acc1.append(a)

plt.plot(acc1)
print(np.mean(acc1))
plt.figure()
acc = []
accuracy = tm.Accuracy()
for batch in test_loader:
  X, y = batch[0].to(device), batch[1].to(device)
  logits = net(X)
  yhat = torch.argmax(logits, axis = 1)
  a = accuracy(yhat.to("cpu"), y.to("cpu"))
  acc.append(a)

plt.plot(acc)
print(np.mean(acc))
plt.figure()


From the data above we can conclude that CNN has a better accuracy than NN.